In [1]:
import os
import rasterio
from rasterio.mask import mask
from rasterio import features
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
from shapely.geometry import shape, Point
import heapq
import time

In [2]:
%matplotlib inline

In [3]:
file_path = r'C:\Users\somadder\Thesis\CP\CP.tif'

In [4]:
current_dir = os.path.dirname(file_path)
os.chdir(current_dir)

In [5]:
contours = gpd.read_file("cp_contour_10m_poly.shp")

In [11]:
contours

,ContourMin,ContourMax,Shape_Leng,Shape_Area,ORIG_FID,geometry,line,slope_min,slope_max,slope_avg
0,1417.501099,2857.220459,388800.000000,9.321727e+09,1,"POLYGON ((680641.181 3988219.932, 680641.181 4...","LINESTRING (680641.181 3988219.932, 680641.181...",NaN,NaN,NaN
1,1420.000000,2857.220459,390011.708000,9.321713e+09,2,"POLYGON ((680641.181 3988219.932, 680641.181 4...","MULTILINESTRING ((680641.181 3988219.932, 6806...",0.327385,42.092430,10.098191
2,1430.000000,2857.220459,397854.840221,9.320435e+09,3,"POLYGON ((680641.181 3988219.932, 680641.181 4...","MULTILINESTRING ((680641.181 3988219.932, 6806...",0.318004,55.774082,12.190907
3,1440.000000,2857.220459,437536.394345,9.313980e+09,4,"POLYGON ((680641.181 3988219.932, 680641.181 4...","MULTILINESTRING ((680641.181 3988219.932, 6806...",0.016430,55.774082,10.098788
4,1440.000000,2857.220459,437536.394345,9.313980e+09,4,"POLYGON ((685985.064 4084513.814, 685977.626 4...","LINESTRING (685985.064 4084513.814, 685977.626...",0.025945,0.249543,0.128244
...,...,...,...,...,...,...,...,...,...,...
28371,2840.000000,2857.220459,5647.878451,4.385666e+05,144,"POLYGON ((692346.181 3997059.333, 692342.321 3...","LINESTRING (692346.181 3997059.333, 692342.321...",4.359091,15.310048,9.818947
28372,2840.000000,2857.220459,5647.878451,4.385666e+05,144,"POLYGON ((691966.181 3997161.975, 691963.389 3...","LINESTRING (691966.181 3997161.975, 691963.389...",7.989002,23.244967,13.903116
28373,2840.000000,2857.220459,5647.878451,4.385666e+05,144,"POLYGON ((692066.181 3997302.783, 692064.551 3...","LINESTRING (692066.181 3997302.783, 692064.551...",10.371949,25.340107,15.701676
28374,2850.000000,2857.220459,1519.897172,7.257680e+04,145,"POLYGON ((692746.181 3994540.849, 692743.327 3...","LINESTRING (692746.181 3994540.849, 692743.327...",1.345956,34.637825,9.087815


In [7]:
contours['line'] = contours.boundary

In [9]:
with rasterio.open('CP_Slope.tif') as src:
    slope_rast = src.read(1)
    for index, row in contours.iterrows():
        line = row.line
        out_slope, out_trans = mask(src, [line], crop=True, filled=True, nodata=-999)
        out_slope_array = out_slope.flatten()
        only_masked_pixels = np.delete(out_slope_array, np.where(out_slope_array == -999))
        if len(only_masked_pixels) > 0:
            min_slope = np.min(only_masked_pixels)
            max_slope = np.max(only_masked_pixels)
            avg_slope = np.average(only_masked_pixels)
            contours.at[index, 'slope_min'] = min_slope
            contours.at[index, 'slope_max'] = max_slope
            contours.at[index, 'slope_avg'] = avg_slope

In [11]:
with rasterio.open('CP_Slope.tif') as src:
    for index, row in contours.iterrows():
        out_slope, out_trans = mask(src, [row.geometry], crop=True, filled=True, nodata=-999)
        out_slope_array = out_slope.flatten()
        only_masked_pixels = np.delete(out_slope_array, np.where(out_slope_array == -999))
        if len(only_masked_pixels) > 0:
            min_slope = np.min(only_masked_pixels)
            max_slope = np.max(only_masked_pixels)
            avg_slope = np.average(only_masked_pixels)
            contours.at[index, 'slope_min'] = min_slope
            contours.at[index, 'slope_max'] = max_slope
            contours.at[index, 'slope_avg'] = avg_slope

In [12]:
contours = contours.drop('line', axis=1)

In [13]:
contours.to_file('cp_contour_10m_poly.shp')